# ResNet

- Jira: http://jira-msc.rnd.huawei.com/browse/AEE-155
- Base: https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

In [19]:
import torch
import torch.nn as nn
from pyraul.pipeline.train_step import train_step
from pyraul.tools.dataset import Dataset
from pyraul.tools.dumping import dump_weights, gen_cpp_dtVec, size
%matplotlib inline

In [20]:
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

## BasicBlock

In [112]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

## Bottleneck

In [56]:
class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

## ResNet

In [113]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1

        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)

## ResNet Types

In [58]:
def resnet18(**kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    """
    return ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)

def resnet34(**kwargs):
    r"""ResNet-34 moodel from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    """
    return ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)

def resnet50(**kwargs):
    r"""ResNet-50 moodel from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    """
    return ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)

def resnet101(**kwargs):
    r"""ResNet-101 moodel from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    """
    return ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)

def resnet152(**kwargs):
    r"""ResNet-152 moodel from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    """
    return ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)

## Tests

In [59]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = resnet18(num_classes=10)
        self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, x):
        x = self.resnet(x)
        x = self.softmax(x)
        return x

### TestResNet, ResNet18CifarPretrainedModelOneEpochTrain

In [122]:
import torch
from pyraul.tools.dumping import dump_weights, DumpMode
from pyraul.tools.seed import set_seed
import torchvision.transforms as transforms
from pyraul.tools.dataset import Dataset
from pyraul.pipeline import accuracy
from pyraul.pipeline.train_step import train_step

config = {
    "seed": 0,
    "classes": 10,
    "bias": True,
    "batch_size": 50,
    "device": "cuda",
#     "device": "cpu",
    "epochs": 20,
    "sgd": {"lr": 0.05}
}

set_seed(config["seed"])

device = torch.device(config["device"])
model = Classifier()
model = model.to(device)

state = torch.load("checkpoint_10")
model.load_state_dict(state["model_state_dict"])

# dump_weights(model, "83.24.txt", mode=DumpMode.flatten_transpose)
# dump_weights(model, "83.24.txt", mode=DumpMode.transpose_flatten, filter="fc")

ds= Dataset("CIFAR10",
            train_transform=[transforms.Resize(224, interpolation=0), transforms.ToTensor()],
            test_transform=[transforms.Resize(224, interpolation=0), transforms.ToTensor()],
            **config)
optimizer = torch.optim.SGD(model.parameters(), lr=config["sgd"]["lr"])
# optimizer = torch.optim.Adam(model.parameters())
criterion = nn.NLLLoss(reduction="mean")

accuracy_before = accuracy(
        model=model,
        dataloader=ds.test_loader,
        **config,
)

print(accuracy_before)

loss, _, _ = train_step(
                    ds.train_loader, 
                    model,
                    criterion,
                    optimizer,
                    device,
                    print_freq=100,
                    verbose=True,
                    loss_history=True
                )

accuracy_after = accuracy(
    model=model,
    dataloader=ds.test_loader,
    **config,
)
print(accuracy_after)

INFO: Loading CIFAR10 dataset...


Files already downloaded and verified
Files already downloaded and verified
83.24
Step    0/1000	Loss: 0.036086 (0.036086)	Time.step: 0.101 (0.101)	Time.load: 0.026 (0.026)
Step  100/1000	Loss: 0.181785 (0.077186)	Time.step: 0.099 (0.101)	Time.load: 0.023 (0.025)
Step  200/1000	Loss: 0.103403 (0.073088)	Time.step: 0.101 (0.101)	Time.load: 0.024 (0.025)
Step  300/1000	Loss: 0.004846 (0.071017)	Time.step: 0.101 (0.101)	Time.load: 0.025 (0.025)
Step  400/1000	Loss: 0.034619 (0.069301)	Time.step: 0.101 (0.101)	Time.load: 0.024 (0.025)
Step  500/1000	Loss: 0.014463 (0.066271)	Time.step: 0.101 (0.101)	Time.load: 0.024 (0.025)
Step  600/1000	Loss: 0.020965 (0.061124)	Time.step: 0.103 (0.101)	Time.load: 0.024 (0.025)
Step  700/1000	Loss: 0.008074 (0.058939)	Time.step: 0.102 (0.101)	Time.load: 0.024 (0.025)
Step  800/1000	Loss: 0.025178 (0.054667)	Time.step: 0.102 (0.102)	Time.load: 0.025 (0.025)
Step  900/1000	Loss: 0.017559 (0.050749)	Time.step: 0.102 (0.102)	Time.load: 0.024 (0.025)
85.24


In [121]:
print(gen_cpp_dtVec(loss.history[::100], "idealLosses"))

const raul::dtVec idealLosses{3.608587e-02_dt, 1.817846e-01_dt, 1.034031e-01_dt, 4.845553e-03_dt, 3.461881e-02_dt, 1.446302e-02_dt, 2.096494e-02_dt, 8.074160e-03_dt, 2.517764e-02_dt, 1.755940e-02_dt};


### TestResNet, BasicBlockBuilding

In [10]:
inplanes = 10
planes = 20
block = BasicBlock(inplanes, planes)
print(block)
print(size(block))

BasicBlock(
  (conv1): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)
5480


### TestResNet, InputBlockBuilding

In [17]:
class InputBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 =  nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    def __forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        return x
    
block = InputBlock()
print(block)
print(size(block))

InputBlock(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)
9536


### TestResNet, ResNet18Building

In [31]:
block = resnet18(num_classes=10)
print(block)
print(size(block))

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### TestResNet, ResNet18Inference

In [114]:
import os
import numpy as np
from pyraul.tools.dumping import dump_weights, gen_cpp_dtVec, DumpMode 
from pyraul.tools.seed import set_seed

config = {
    "seed": 0,
    "classes": 10
}

set_seed(config["seed"])

model = resnet18(num_classes=config["classes"])
model.eval()

# dump_weights(model, "init.txt", mode=DumpMode.flatten_transpose)
dump_weights(model, "init.txt", mode=DumpMode.transpose_flatten, filter="fc")

tensor_in = torch.from_numpy(np.ones((1, 3, 224, 224), dtype=np.float32)*5)
tensor_out = model(tensor_in)
print(gen_cpp_dtVec(tensor_out.data.flatten(), "golden_tensor_out"))

const raul::dtVec golden_tensor_out{1.403602e+00_dt, 9.141974e-03_dt, -2.788210e+00_dt, 2.950551e+00_dt, -1.179150e+00_dt, -2.007151e+00_dt, -3.162097e-01_dt, -8.549929e-01_dt, 1.268970e+00_dt, 3.833183e-01_dt};


#### Sizes of weights

In [52]:
from functools import reduce

for (name, param) in model.named_parameters():
    if param.requires_grad and "conv" in name:
        print(name, reduce(lambda a,b: a*b, param.shape, 1))

conv1.weight 9408
layer1.0.conv1.weight 36864
layer1.0.conv2.weight 36864
layer1.1.conv1.weight 36864
layer1.1.conv2.weight 36864
layer2.0.conv1.weight 73728
layer2.0.conv2.weight 147456
layer2.1.conv1.weight 147456
layer2.1.conv2.weight 147456
layer3.0.conv1.weight 294912
layer3.0.conv2.weight 589824
layer3.1.conv1.weight 589824
layer3.1.conv2.weight 589824
layer4.0.conv1.weight 1179648
layer4.0.conv2.weight 2359296
layer4.1.conv1.weight 2359296
layer4.1.conv2.weight 2359296


## Traininig

### Up to 80% accuracy

In [ ]:
from pyraul.tools.seed import set_seed
import torchvision.transforms as transforms
from pyraul.tools.dataset import Dataset
from pyraul.pipeline import accuracy
from pyraul.pipeline.train_step import train_step
from pyraul.tools.dumping import save_checkpoint

config = {
    "seed": 0,
    "classes": 10,
    "bias": True,
    "batch_size": 50,
    "device": "cuda",
#     "device": "cpu",
    "epochs": 30,
    "sgd": {"lr": 0.05}
}

set_seed(config["seed"])

device = torch.device(config["device"])
model = Classifier()
model = model.to(device)

ds= Dataset("CIFAR10",
            train_transform=[transforms.Resize(224, interpolation=0), transforms.ToTensor()],
            test_transform=[transforms.Resize(224, interpolation=0), transforms.ToTensor()],
            **config)
# optimizer = torch.optim.SGD(model.parameters(), lr=config["sgd"]["lr"])
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.NLLLoss(reduction="mean")

accuracy_before = accuracy(
        model=model,
        dataloader=ds.test_loader,
        **config,
)

print(accuracy_before)

acc_history=[accuracy_before]
loss_history=[]

epochs = config["epochs"]
for epoch in range(1, epochs+1):
        print(f"Epoch: {epoch:03}/{epochs:03}")
        loss, _, _ = train_step(
                            ds.train_loader, 
                            model,
                            criterion,
                            optimizer,
                            device,
                            print_freq=100,
                            verbose=True,
                            loss_history=True
                        )

        loss_history += loss.history
        
        accuracy_after = accuracy(
            model=model,
            dataloader=ds.test_loader,
            **config,
        )
        print(accuracy_after)
        acc_history.append(accuracy_after)
        
        save_checkpoint({"epoch": epoch,
                         "acc": accuracy_after, 
                         "acc_history": acc_history,
                         "loss_history": loss_history,
                         "model_state_dict": model.state_dict(),
                         "optimizer_state_dict": optimizer.state_dict()
                        }, filename=f"checkpoint_{epoch}")

INFO: Loading CIFAR10 dataset...


Files already downloaded and verified
Files already downloaded and verified
10.0
Epoch: 001/030
Step    0/1000	Loss: 2.415540 (2.415540)	Time.step: 0.103 (0.103)	Time.load: 0.025 (0.025)
Step  100/1000	Loss: 1.704273 (1.926329)	Time.step: 0.102 (0.103)	Time.load: 0.024 (0.025)
Step  200/1000	Loss: 1.557923 (1.790768)	Time.step: 0.104 (0.103)	Time.load: 0.025 (0.025)
Step  300/1000	Loss: 1.519650 (1.702788)	Time.step: 0.104 (0.103)	Time.load: 0.025 (0.025)
Step  400/1000	Loss: 1.504326 (1.630857)	Time.step: 0.104 (0.103)	Time.load: 0.025 (0.025)
Step  500/1000	Loss: 0.951241 (1.560308)	Time.step: 0.104 (0.103)	Time.load: 0.025 (0.025)
Step  600/1000	Loss: 1.104180 (1.498991)	Time.step: 0.103 (0.104)	Time.load: 0.024 (0.025)
Step  700/1000	Loss: 1.052298 (1.450008)	Time.step: 0.104 (0.104)	Time.load: 0.025 (0.025)
Step  800/1000	Loss: 0.908380 (1.403901)	Time.step: 0.104 (0.104)	Time.load: 0.024 (0.025)
Step  900/1000	Loss: 0.679503 (1.363849)	Time.step: 0.104 (0.104)	Time.load: 0.024 (0

#### Helper

In [134]:
a=[
    "input::bn1::Biases",
"input::bn1::MeanEval",
"input::bn1::VarianceEval",
"input::bn1::Weights",
"layer1::block0::bn1::Biases",
"layer1::block0::bn1::MeanEval",
"layer1::block0::bn1::VarianceEval",
"layer1::block0::bn1::Weights",
"layer1::block0::bn2::Biases",
"layer1::block0::bn2::MeanEval",
"layer1::block0::bn2::VarianceEval",
"layer1::block0::bn2::Weights",
"layer1::block1::bn1::Biases",
"layer1::block1::bn1::MeanEval",
"layer1::block1::bn1::VarianceEval",
"layer1::block1::bn1::Weights",
"layer1::block1::bn2::Biases",
"layer1::block1::bn2::MeanEval",
"layer1::block1::bn2::VarianceEval",
"layer1::block1::bn2::Weights",
"layer2::block0::bn1::Biases",
"layer2::block0::bn1::MeanEval",
"layer2::block0::bn1::VarianceEval",
"layer2::block0::bn1::Weights",
"layer2::block0::bn2::Biases",
"layer2::block0::bn2::MeanEval",
"layer2::block0::bn2::VarianceEval",
"layer2::block0::bn2::Weights",
"layer2::block0::downsample::bn1::Biases",
"layer2::block0::downsample::bn1::MeanEval",
"layer2::block0::downsample::bn1::VarianceEval",
"layer2::block0::downsample::bn1::Weights",
"layer2::block1::bn1::Biases",
"layer2::block1::bn1::MeanEval",
"layer2::block1::bn1::VarianceEval",
"layer2::block1::bn1::Weights",
"layer2::block1::bn2::Biases",
"layer2::block1::bn2::MeanEval",
"layer2::block1::bn2::VarianceEval",
"layer2::block1::bn2::Weights",
"layer3::block0::bn1::Biases",
"layer3::block0::bn1::MeanEval",
"layer3::block0::bn1::VarianceEval",
"layer3::block0::bn1::Weights",
"layer3::block0::bn2::Biases",
"layer3::block0::bn2::MeanEval",
"layer3::block0::bn2::VarianceEval",
"layer3::block0::bn2::Weights",
"layer3::block0::downsample::bn1::Biases",
"layer3::block0::downsample::bn1::MeanEval",
"layer3::block0::downsample::bn1::VarianceEval",
"layer3::block0::downsample::bn1::Weights",
"layer3::block1::bn1::Biases",
"layer3::block1::bn1::MeanEval",
"layer3::block1::bn1::VarianceEval",
"layer3::block1::bn1::Weights",
"layer3::block1::bn2::Biases",
"layer3::block1::bn2::MeanEval",
"layer3::block1::bn2::VarianceEval",
"layer3::block1::bn2::Weights",
"layer4::block0::bn1::Biases",
"layer4::block0::bn1::MeanEval",
"layer4::block0::bn1::VarianceEval",
"layer4::block0::bn1::Weights",
"layer4::block0::bn2::Biases",
"layer4::block0::bn2::MeanEval",
"layer4::block0::bn2::VarianceEval",
"layer4::block0::bn2::Weights",
"layer4::block0::downsample::bn1::Biases",
"layer4::block0::downsample::bn1::MeanEval",
"layer4::block0::downsample::bn1::VarianceEval",
"layer4::block0::downsample::bn1::Weights",
"layer4::block1::bn1::Biases",
"layer4::block1::bn1::MeanEval",
"layer4::block1::bn1::VarianceEval",
"layer4::block1::bn1::Weights",
"layer4::block1::bn2::Biases",
"layer4::block1::bn2::MeanEval",
"layer4::block1::bn2::VarianceEval",
"layer4::block1::bn2::Weights"
]

b = [
  "bn1.bias",
"bn1.running_mean",
"bn1.running_var",
"bn1.weight",
"layer1.0.bn1.bias",
"layer1.0.bn1.running_mean",
"layer1.0.bn1.running_var",
"layer1.0.bn1.weight",
"layer1.0.bn2.bias",
"layer1.0.bn2.running_mean",
"layer1.0.bn2.running_var",
"layer1.0.bn2.weight",
"layer1.1.bn1.bias",
"layer1.1.bn1.running_mean",
"layer1.1.bn1.running_var",
"layer1.1.bn1.weight",
"layer1.1.bn2.bias",
"layer1.1.bn2.running_mean",
"layer1.1.bn2.running_var",
"layer1.1.bn2.weight",
"layer2.0.bn1.bias",
"layer2.0.bn1.running_mean",
"layer2.0.bn1.running_var",
"layer2.0.bn1.weight",
"layer2.0.bn2.bias",
"layer2.0.bn2.running_mean",
"layer2.0.bn2.running_var",
"layer2.0.bn2.weight",
"layer2.0.downsample.1.bias",
"layer2.0.downsample.1.running_mean",
"layer2.0.downsample.1.running_var",
"layer2.0.downsample.1.weight",
"layer2.1.bn1.bias",
"layer2.1.bn1.running_mean",
"layer2.1.bn1.running_var",
"layer2.1.bn1.weight",
"layer2.1.bn2.bias",
"layer2.1.bn2.running_mean",
"layer2.1.bn2.running_var",
"layer2.1.bn2.weight",
"layer3.0.bn1.bias",
"layer3.0.bn1.running_mean",
"layer3.0.bn1.running_var",
"layer3.0.bn1.weight",
"layer3.0.bn2.bias",
"layer3.0.bn2.running_mean",
"layer3.0.bn2.running_var",
"layer3.0.bn2.weight",
"layer3.0.downsample.1.bias",
"layer3.0.downsample.1.running_mean",
"layer3.0.downsample.1.running_var",
"layer3.0.downsample.1.weight",
"layer3.1.bn1.bias",
"layer3.1.bn1.running_mean",
"layer3.1.bn1.running_var",
"layer3.1.bn1.weight",
"layer3.1.bn2.bias",
"layer3.1.bn2.running_mean",
"layer3.1.bn2.running_var",
"layer3.1.bn2.weight",
"layer4.0.bn1.bias",
"layer4.0.bn1.running_mean",
"layer4.0.bn1.running_var",
"layer4.0.bn1.weight",
"layer4.0.bn2.bias",
"layer4.0.bn2.running_mean",
"layer4.0.bn2.running_var",
"layer4.0.bn2.weight",
"layer4.0.downsample.1.bias",
"layer4.0.downsample.1.running_mean",
"layer4.0.downsample.1.running_var",
"layer4.0.downsample.1.weight",
"layer4.1.bn1.bias",
"layer4.1.bn1.running_mean",
"layer4.1.bn1.running_var",
"layer4.1.bn1.weight",
"layer4.1.bn2.bias",
"layer4.1.bn2.running_mean",
"layer4.1.bn2.running_var",
"layer4.1.bn2.weight"  
]

from functools import reduce
data = {name: reduce(lambda x,y: x*y, param.shape, 1) for (name, param) in model.state_dict().items()}

for x, y in zip(a,b):
    print(f"{{\"{x}\", {{\"{y}\", {data['resnet.'+y]}}}}},")

{"input::bn1::Biases", {"bn1.bias", 64}},
{"input::bn1::MeanEval", {"bn1.running_mean", 64}},
{"input::bn1::VarianceEval", {"bn1.running_var", 64}},
{"input::bn1::Weights", {"bn1.weight", 64}},
{"layer1::block0::bn1::Biases", {"layer1.0.bn1.bias", 64}},
{"layer1::block0::bn1::MeanEval", {"layer1.0.bn1.running_mean", 64}},
{"layer1::block0::bn1::VarianceEval", {"layer1.0.bn1.running_var", 64}},
{"layer1::block0::bn1::Weights", {"layer1.0.bn1.weight", 64}},
{"layer1::block0::bn2::Biases", {"layer1.0.bn2.bias", 64}},
{"layer1::block0::bn2::MeanEval", {"layer1.0.bn2.running_mean", 64}},
{"layer1::block0::bn2::VarianceEval", {"layer1.0.bn2.running_var", 64}},
{"layer1::block0::bn2::Weights", {"layer1.0.bn2.weight", 64}},
{"layer1::block1::bn1::Biases", {"layer1.1.bn1.bias", 64}},
{"layer1::block1::bn1::MeanEval", {"layer1.1.bn1.running_mean", 64}},
{"layer1::block1::bn1::VarianceEval", {"layer1.1.bn1.running_var", 64}},
{"layer1::block1::bn1::Weights", {"layer1.1.bn1.weight", 64}},
{"layer1

In [2]:
%load_ext watermark
%watermark -d -u -v -iv

torch 1.4.0
last updated: 2020-04-22 

CPython 3.8.0
IPython 7.13.0
